In [ ]:
# 필요한 라이브러리 import
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset

In [ ]:
# 하이퍼 파라미터 설정
num_epochs = 7
num_classes = 2
batch_size = 128
learning_rate = 0.0005

# GPU 설정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# GPU 설정 확인
torch.cuda.is_available()

In [ ]:
# 데이터 불러오기
labels = pd.read_csv('../input/train_labels.csv')
sub = pd.read_csv('../input/sample_submission.csv')

# 파일 경로 설정
train_path = '../input/train/'
test_path = '../input/test/'

In [ ]:
# Validation 구간 20개로 나눔 train : val = 19 : 1
train, val = train_test_split(labels, stratify=labels.label, test_size=0.05)
len(train), len(val)

**Simple custom generator**

In [ ]:
# 데이터셋 오버라이딩. 이것을 해줘야 속도가 빨라진다고 한다.
# len, getitem 오버라이딩
class MyDataset(Dataset):
    def __init__(self, df_data, data_dir = './', transform=None):
        super().__init__()
        self.df = df_data.values
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_name,label = self.df[index]
        img_path = os.path.join(self.data_dir, img_name+'.tif')
        image = cv2.imread(img_path)
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [ ]:
# train 데이터 전처리
trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.RandomHorizontalFlip(), 
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20), 
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

# validation 데이터 전처리
trans_valid = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

# 데이터 셋 설정
dataset_train = MyDataset(df_data=train, data_dir=train_path, transform=trans_train)
dataset_valid = MyDataset(df_data=val, data_dir=train_path, transform=trans_valid)

# 데이터 셋에서 배치 단위로 불러옴
loader_train = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=batch_size//2, shuffle=False, num_workers=0)

**Model**

In [ ]:
# 초기에 사용했던 CNN 신경망
# 5개의 conv layer 와 1개의 fc layer 사용
# conv layer 마다 batch norm과 활성함수 적용
# 활성 함수 로는 leaky ReLU 사용
# 활성 함수 적용 이후 pooling 적용
# Max Pooling 을 사용함
# 모든 과정을 거쳐서 하나의 벡터로 만든후 fc layer 통과
class SimpleCNN(nn.Module):
    def __init__(self):
        # ancestor constructor call
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avg = nn.AvgPool2d(8)
        self.fc = nn.Linear(512 * 1 * 1, 2) # !!!
    def forward(self, x):
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x))))
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x))))
        x = self.pool(F.leaky_relu(self.bn4(self.conv4(x))))
        x = self.pool(F.leaky_relu(self.bn5(self.conv5(x))))
        x = self.avg(x)
        x = x.view(-1, 512 * 1 * 1) # !!!
        x = self.fc(x)
        return x

In [ ]:
# 우리가 사용한 신경망 모델
# torchvision 에서 제공하는 resnet50을 사용함
# resnet50의 구조는 보고서에 첨부
model = torchvision.models.resnet50().to(device)

In [ ]:
# 손실 함수로 교차 엔트로피 사용
criterion = nn.CrossEntropyLoss()

# 최적화 방법으로 RMSProp 사용
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay = 0.0001)

# 두 가지 방법의 자세한 내용은 보고서에 첨부

In [ ]:
# 모델 학습 단계
total_step = len(loader_train)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(loader_train):
        images = images.to(device)
        labels = labels.to(device)
        
        # 순전파
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 역전파 및 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

**Accuracy Check**

In [ ]:
# 모델 검증 단계
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
          
    print('Test Accuracy of the model on the 22003 test images: {} %'.format(100 * correct / total))

# checkpoint 저장
torch.save(model.state_dict(), 'model.ckpt')

**CSV submission**

In [ ]:
# validation 데이터 셋 설정 및 불러오기
dataset_valid = MyDataset(df_data=sub, data_dir=test_path, transform=trans_valid)
loader_test = DataLoader(dataset = dataset_valid, batch_size=32, shuffle=False, num_workers=0)

In [ ]:
# validation 단계
model.eval()

preds = []
for batch_i, (data, target) in enumerate(loader_test):
    data, target = data.cuda(), target.cuda()
    output = model(data)

    pr = output[:,1].detach().cpu().numpy()
    for i in pr:
        preds.append(i)
        
# 제출 파일 생성
sub.shape, len(preds)
sub['label'] = preds
sub.to_csv('submit_KMUAI_2.csv', index=False)